In [1]:
# 定义用于演示常量表达式优化的函数
def my_func():
    # 1. 数值常量乘法：编译时会预先计算 24*60 的结果（1440）并缓存，运行时直接使用缓存值
    a = 24 * 60  
    # 2. 元组（不可变序列）乘法：编译时预计算 (1,2)*5 的展开结果，避免运行时重复计算
    b = (1, 2) * 5  
    # 3. 字符串（不可变序列）乘法：编译时预计算 'abc'*3 的拼接结果（'abcabcabc'）并缓存
    c = 'abc' * 3  
    # 4. 字符串乘法：'ab'*11 结果长度为22，超过Python常量折叠的默认长度阈值（20），不进行预计算
    d = 'ab' * 11  
    # 5. 字符串乘法：'the quick brown fox'*10 结果长度过长，不符合预计算条件，不缓存结果
    e = 'the quick brown fox' * 10  
    # 6. 列表（可变序列）乘法：列表是可变对象，无法在编译期确定其最终状态，不满足“常量”属性，不预计算
    f = [1, 2] * 5  

In [2]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababab',
 'the quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown fox',
 1,
 2,
 5)

In [3]:
# 定义包含列表成员测试的函数
def my_func(e):
    # 列表 [1,2,3] 用于成员测试：
    # 编译时会自动将可变的列表转换为不可变的元组（(1,2,3)）
    # 优化原因：成员测试仅需“读取元素”操作，无需修改序列；元组比列表内存占用更少、访问效率更高，且不影响测试结果
    if e in [1, 2, 3]:
        pass  # 占位语句，无实际业务逻辑，仅用于演示成员测试场景

In [4]:
# 查看函数编译后的常量缓存池，验证优化结果
my_func.__code__.co_consts

(None, (1, 2, 3))

In [5]:
# 定义包含列表成员测试的函数
def my_func(e):
    # 列表 [1,2,3] 用于成员测试：
    # 编译时会自动将可变的列表转换为不可变的元组（(1,2,3)）
    # 优化原因：成员测试仅需“读取元素”操作，无需修改序列；元组比列表内存占用更少、访问效率更高，且不影响测试结果
    if e in {1, 2, 3}:
        pass  # 占位语句，无实际业务逻辑，仅用于演示成员测试场景

In [7]:
# 查看函数编译后的常量缓存池，验证优化结果
my_func.__code__.co_consts

(None, frozenset({1, 2, 3}))

In [8]:
# 导入所需模块：string（生成大小写字母序列）、time（高精度计时，用于统计耗时）
import string
import time

all_letters = string.ascii_letters
all_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [9]:
# 生成三种包含所有大小写字母的容器（用于对比不同容器的成员测试性能）
# 1. 列表：可变序列，成员测试需遍历所有元素（时间复杂度 O(n)，n为元素数量）
char_list = list(string.ascii_letters)
# 2. 元组：不可变序列，成员测试同样需遍历元素（时间复杂度 O(n)）
char_tuple = tuple(string.ascii_letters)
# 3. 集合：基于哈希表实现，成员测试通过哈希值直接定位（时间复杂度 O(1)，无需遍历）
char_set = set(string.ascii_letters)

In [10]:
# 打印三种容器的内容（验证元素一致性，确保测试公平性，仅作演示用）
print(char_list)
print()  # 空行分隔，提升输出可读性
print(char_tuple)
print()
print(char_set)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')

{'i', 'h', 'M', 'C', 'r', 'c', 'O', 'V', 'U', 'R', 'd', 'e', 'A', 'X', 'q', 'K', 'm', 'v', 'Z', 'W', 'g', 'j', 'L', 'l', 'u', 'o', 'D', 'Y', 'P', 'G', 'f', 'E', 'S', 'p', 'w', 'y', 'F', 't', 'z', 's', 'n', 'I', 'N', 'x', 'J', 'b', 'B', 'a', 'k', 'H', 'T', 'Q'}


In [11]:
# 定义成员测试函数：模拟大量重复的成员检查场景
# 参数 n：测试次数（此处设为1000万次，确保不同容器的耗时差异可明显观测）
# 参数 container：待测试的容器（列表/元组/集合）
def membership_test(n, container):
    for i in range(n):
        # 核心测试操作：检查字符 'p' 是否在容器中
        if 'p' in container:
            pass  # 无实际业务逻辑，仅保留成员测试操作，避免其他代码干扰计时

In [12]:
# 1. 测试列表的成员测试耗时
start = time.perf_counter()  # 记录测试开始时间（高精度计时，比time.time()更适合性能测试）
membership_test(10000000, char_list)  # 执行1000万次列表成员测试
end = time.perf_counter()  # 记录测试结束时间
# 输出列表测试耗时：由于需遍历所有元素，耗时最长（示例输出：2.6035404184015434秒）
print('列表成员测试耗时: ', end - start)

列表成员测试耗时:  2.3238497079999547


In [13]:
# 2. 测试元组的成员测试耗时
start = time.perf_counter()
membership_test(10000000, char_tuple)  # 执行1000万次元组成员测试
end = time.perf_counter()
# 输出元组测试耗时：与列表遍历逻辑一致，耗时接近（示例输出：2.602491734651276秒）
print('元组成员测试耗时: ', end - start)

元组成员测试耗时:  2.4721367499999474


In [14]:
# 3. 测试集合的成员测试耗时
start = time.perf_counter()
membership_test(10000000, char_set)  # 执行1000万次集合成员测试
end = time.perf_counter()
# 输出集合测试耗时：基于哈希表查找，效率极高，耗时仅为列表/元组的1/7左右（示例输出：0.3743007599607324秒）
print('集合成员测试耗时: ', end - start)

集合成员测试耗时:  0.26347491699999637


- 核心逻辑：我们模拟 1000 万次成员测试，对比列表、元组、集合的耗时；
- 结果规律：
    1. 列表和元组耗时几乎一样（约 2.3 秒）—— 因为都是 “遍历查找”，时间复杂度 O (n)；
    2. 集合仅需约 0.26 秒，是列表 / 元组的 1/7—— 因为哈希表直接定位，时间复杂度 O (1)；
- 实战结论：写成员测试时，只要场景允许（不需要有序、不需要重复元素），优先用集合`{}`，而不是列表`[]`或元组`()`。

# 总结

1. 验证编译期优化：可以通过`函数.__code__.co_consts`查看常量缓存池，确认 Python 是否做了预计算、对象替换；
2. 常量表达式优化：Python 自动帮我们优化 “不可变、短长度” 的常量计算，不用手动改，但要知道 “可变对象（列表）、长序列” 不会被优化；
3. 成员测试优化：主动用集合替代列表 / 元组，是最直接的 “手写优化”，能大幅提升性能。